<h1>Задание 3. Численное решение интегрального уравнения Фредгольма второго рода</h1>
<h2 align = "center"><font color="#6f8600">Вариант 9</font></h2>
<p>Будем искать решение следующей задачи:</p>
$u(x) + 0.1\int\limits_{0}^{1} \sin(x(0.5+y^{2}))u(y)dy = x + 0.1$, применяя составную формулу трапеций

<h5>Составная формула трапеций</h5>
<p>Заменим интеграл в условии на квадратурную сумму по формуле трапеций:</p>
$\int\limits_{0}^{1} f(x)dx \approx \frac{h}{2}(f(0)+f(1)+2\sum\limits_{i=1}^{n-1}f(x_{i}))$, где h - длина отрезка $ [x_{i-1}, x_{i}]$
<p>Тогда получим уравнение: $ u^{n}(x) - (\frac{h}{2}(H(x,x_{0})+H(x,x_{n}))+\sum\limits_{k=1}^{n-1}hH(x,x_{k}))u^{n}(x_{k}) = x + 0.1$</p>
<p>Здесь $x_{i} = 0 + ih = ih $, где $ h = \frac{1-0}{n} = \frac{1}{n}, i = 0, 1, \dots, n$</p>

<h5>Реализация</h5>

In [44]:
import sympy as sp
import math
import pandas as pd
import numpy as np

# Определим некоторые постоянные
eps = 0.00001
a = 0
b = 1
n = 1
err = 1

# Определим функции f и H
x, y = sp.symbols("x y")
f = sp.simplify(x+0.1)
H = sp.simplify(-0.1*sp.sin(x*(0.5+y**2)))

# Определим таблицу результатов
res_n_left = []
res_n_middle = []
res_n_right = []

while True:
#     Определим узлы и коэффициенты для формулы трапеции
    h = 1/n
    [x_k, A] = findNodesAndCoeff(h, n)
    
#     Определим СЛАУ вида Dz = g, благодаря которой можно вычислить решение
    d = np.zeros((n+1, n+1))
    g = np.zeros(n+1)
    for i in range(0, n+1):
        for j in range(0, n+1):
            d[i,j] = kron(i,j) - A[j]*H.subs([(x, x_k[i]), (y, x_k[j])])
        g[i] = f.subs(x, x_k[i])
    
#     Находим решение СЛАУ    
    z = countSLAE(d, g)
    
#     Вычислим u_n(x)
    u_found = countU(z, x_k, A, f, H, n)
    
    res_n_left.append(u_found.subs(x, 0))
    res_n_middle.append(u_found.subs(x, 0.5))
    res_n_right.append(u_found.subs(x, 1))
    
    if n == 1:
        n = n+1
    else:
        err = max(abs(res_n_left[-1] - res_n_left[-2]), abs(res_n_middle[-1] - res_n_middle[-2]), abs(res_n_right[-1] - res_n_right[-2]))
        n = n+1
    if err < eps:
        break

<h5>Функция печати таблицы результатов</h5>

In [45]:
df = pd.DataFrame({ 
                   'u_n(0)': res_n_left[:],
                   'u_n(1/2)': res_n_middle[:], 
                   'u_n(1)': res_n_right[:] 
                  })
df

,u_n(0),u_n(1/2),u_n(1)
0,0.100000000000000,0.563131650783018,1.04546077806636
1,0.100000000000000,0.570979280142436,1.05308029778932
2,0.100000000000000,0.572367962496634,1.05399966425909
3,0.100000000000000,0.572846995489873,1.05428090390227
4,0.100000000000000,0.573067412226857,1.05440404118913
5,0.100000000000000,0.573186790523073,1.05446907874007
6,0.100000000000000,0.573258650092701,1.05450766717972
7,0.100000000000000,0.573305240403207,1.05453246078842
8,0.100000000000000,0.573337160054986,1.05454934490150
9,0.100000000000000,0.573359980732578,1.05456136503944


<h5>Погрешность последнего шага</h5>

In [70]:
print(f"В точке x = 0 погрешность: {res_n_left[-1] - res_n_left[-2]}")
print(f"В точке x = 1/2 погрешность: {res_n_middle[-1] - res_n_middle[-2]}")
print(f"В точке x = 1 погрешность: {res_n_right[-1] - res_n_right[-2]}")

В точке x = 0 погрешность: 0
В точке x = 1/2 погрешность: 0.00000998592523959729
В точке x = 1 погрешность: 0.00000522218628495885


<h5>Функция для поиска узлов и коэффициентов составной формулы трапеций</h5>

In [13]:
def findNodesAndCoeff(h, n):
    nodes = []
    coeff = []
    for i in range(0, n+1):
        coeff.append(h)
        nodes.append(i*h)
    coeff[0] = coeff[0]/2
    coeff[n] = coeff[n]/2
    return [nodes, coeff]

<h5>Символ Кронекера</h5>

In [14]:
def kron(i,j):
    if i == j:
        return 1
    else:
        return 0

<h5>Функция для решения СЛАУ</h5>

In [16]:
def countSLAE(D, g):
    c = np.linalg.solve(D, g)
    return c

<h5>Функция для нахождения решения исходного уравнения, после вычисления решения СЛАУ</h5>

In [19]:
def countU(z, x_k, A, f, H, n):
    u = sp.simplify(f)
    for i in range(0, n+1):
        u = sp.simplify(u+ A[i]*H.subs(y, x_k[i])*z[i])
    return u